# Aufgabe 29: Lösungsvorschlag

In [1]:
using LinearAlgebra
using Random

Unveränderter generische Bisekionsalgorithmus aus der Vorlesung:

In [2]:
struct Interval # Datenstruktur für Intervall [a,b] mit zusätzlichen Daten
    a::Number
    b::Number
    s₀::Number # Speicher für s(a): verhindert unnötige Neuauswertungen
    s₁::Number # Speicher für s(b)
end

Base.show(io::IO, ival::Interval) = Base.show(io, showival(ival)) # `showival` muss spezifiziert werden

Interval(a, b, s::Function) = Interval(a, b, s(a), s(b)) # initialisiere Intervall [a,b] mit Daten s(a), s(b)
length(ival::Interval) = ival.b - ival.a                 # Länge 
midpnt(ival::Interval) = (ival.a + ival.b)/2             # Mittelpunkt 
tokeep(ival::Interval) = ival.s₁ != ival.s₀              # behalte, solange s(a)≠s(b) 

function split(ival::Interval, s::Function)              # zerlege Intervall beim Mittelpunkt
    m = midpnt(ival)
    sₘ = s(m)
    return [Interval(ival.a, m, ival.s₀, sₘ), Interval(m, ival.b, sₘ, ival.s₁)] 
end

function bisect(s::Function, a, b, tol)  
    tol = max(tol, eps(float(maximum(abs.([a,b]))))) # tol ≥ Auflösung von IEEE754 in [a,b]
    list = [ Interval(a, b, s) ]                     # initialisiere Liste der Intervalle 
    while (todo = list[length.(list) .> tol]) != []  # gibt es noch zu bearbeitende Intervalle?
        for ival in todo                             # für all diese:
            setdiff!(list,[ival])                    #    entferne das Intervall aus der Liste
            parts = split(ival, s)                   #    unterteile das Intervall
            push!(list, parts[tokeep.(parts)]...)    #    füge die erfolgreichen Teile hinzu
        end
    end
    return list
end;

In [3]:
Random.seed!(0815)
m = 5000; A = Symmetric(randn(m,m));
T = hessenberg(A).H;

In [4]:
negnum(T::SymTridiagonal, μ) = sum(ldlt(T, shift=-μ).d .< 0)    # Anzahl EW < μ

# Suchfunktion: wechselt Boole'schen Wert genau bei λₙ
n = 100
suchfunc(μ) = negnum(T, μ) < n ? 1 : 0;

In [5]:
# Festlegung der Ausgabe
showival(ival::Interval) = [ival.a, ival.b]; # zeige nur das Interval

In [6]:
function eiginterval(A:: Union{Symmetric,SymTridiagonal}) # aus Gerschgorin, Aufgabe 28
    m, = size(A)
    r = abs.(A - Diagonal(A))*ones(m)
    return [minimum(diag(A)-r), maximum(diag(A)+r)]
end;

In [7]:
a, b = eiginterval(T)
bisect(suchfunc, a, b, opnorm(T, Inf)*eps())

1-element Vector{Interval}:
 [-126.70723085995971, -126.70723085995968]

In [8]:
λ = sort(eigvals(T)); λ[100] # Kontrolle... 

-126.70723085995928